In [20]:
import pandas as pd
from datetime import datetime

from IPython.display import display
from IPython.display import Markdown

# 先確認目前目錄，from {檔案路徑/.py} import {function}
from src.utils import read_config, OracleAgent

from llama_index.core.schema import Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.chroma import ChromaVectorStore

import chromadb

取得根目錄


In [ ]:
import os
os.getcwd()

In [22]:
info_path = '.env/info.json'
info = read_config(info_path)

In [4]:
oracle_agent = OracleAgent(info['DW_conn_info'])

In [5]:
query = """
select rowid, dashboard_name, report_name, page_name, pic_name, llm as description
from bi_dashboard
"""

df_dashboard = oracle_agent.read_table(query=query)

In [6]:
df_dashboard.head(2)

,rowid,dashboard_name,report_name,page_name,pic_name,description
0,AAi0soABZAAAACLAAC,投控經營分析,華紙-營運績效分析月報,紙機生產狀況,紙機生產狀況-得率趨勢圖.jpg,這張圖表顯示的是 2024 年 1 月到 5 月的月度效率數據。圖表上的數字代表各個月的效率...
1,AAi0soABZAAAACLAAF,投控經營分析,華紙-營運績效分析月報,號機直接法損益,號機直接法損益-完整畫面.jpg,這張圖片包含了一份營運績效分析月報，其中包含以下指標：\n\n**成本**:\n* 銷貨成本...


In [7]:
Markdown(df_dashboard[df_dashboard['rowid'] == 'AAi0soABZAAAACUAAH'].description.values[0])

這張圖表顯示了生產製造成本的組成，可以帮助你了解不同項目在生產成本中所占的比例。

**主要分析指標：**

* **生產製造成本的總金額：** 顯示了總的生產製造成本。
* **各項成本的金額：** 顯示了不同項目在生產成本中的具體金額。
* **各項成本的百分比：** 顯示了不同項目在生產成本中所占的比例。 
* **累積百分比：** 顯示了累積的成本比例，可以帮助你快速了解不同項目的成本佔比。

**可以查詢的指標：**

* **各項成本的金額和百分比：** 您可以查看不同項目的成本金額和百分比，例如：
    * **原料：** 顯示了原材料的成本。
    * **電力：** 顯示了電力的成本。
    * **製造成本：** 顯示了生產過程中的成本。
    * **人工：** 顯示了人工成本。
    * **其他：** 顯示了其他成本。
* **累積百分比：** 您可以查看累積百分比，例如：
    * **原料的累積百分比：** 顯示了原材料成本佔總成本的比例。
    * **原料和電力的累積百分比：** 顯示了原材料和電力成本佔總成本的比例。
    * **所有成本的累積百分比：** 顯示了所有成本佔總成本的比例，應該為 100%。

**用途：**

這張圖表可以帮助您：

* 了解生產製造成本的組成。
* 分析不同項目的成本佔比。
* 找出成本的主要來源。
* 評估成本控制的成效。

**注意事項：**

* 圖表中顯示的數據是特定時間點的數據，可能不代表長期趨勢。
* 圖表中沒有提供成本變化的趨勢信息。
* 圖表中沒有提供成本控制措施的信息。


In [9]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embeddings = HuggingFaceEmbedding(model_name=model_name)

In [13]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = chroma_client.get_or_create_collection("DashboardQA")

In [15]:
# list comprehension
def df_to_documents(df):
   return [Document(text=row['description'], metadata={col: row[col] for col in df.columns if col != 'descriptoin'}) for _, row in df.iterrows()]

In [17]:
documents = df_to_documents(df_dashboard)

In [21]:
# store in vectordb
Settings.chunk_size = 1024
Settings.chunk_overlap = 50
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_store_index = VectorStoreIndex.from_documents(
   documents=documents,
   storage_context=storage_context,
   embed_model=embeddings
)

In [23]:
# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("DashboardQA")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
   vector_store,
   embed_model=embeddings,
)

In [24]:
# Query Data from the persisted index
query_engine = index.as_retriever(similarity_top_k=3)
response = query_engine.retrieve("""              
哪張表格包含
淨利、稅前淨利、稅後淨利
""")
display(Markdown(f"{response[0]}"))

Node ID: 7d4d2ccd-3a81-4b7d-831f-64a1eadd3a6a
Text: 這張圖片顯示了一份華紙營運績效分析月報，包含了各事業部的相關分析指標。表格中顯示了以下指標：  **本月** - 銷售收入 -
銷售成本 - 銷售毛利 - 營業利益 - 稅前淨利  **全年累計** - 銷售收入 - 銷售成本 - 銷售毛利 - 營業利益 - 稅前淨利
- 年度預算 - 達成率  表格中的商品類別包含： - 等等(造紙業) - 久違(紙) - 花蓮(紙) - 台紙(紙) - 智慧(紙) -
P板(紙) - 昆山紙業(CNY) - 昆林紙業(CNY) - 纖維製品(CNY) - 合紙(紙)
您可以根據您的需要，查詢您感興趣的指標。
Score:  0.232


In [25]:
response

[NodeWithScore(node=TextNode(id_='7d4d2ccd-3a81-4b7d-831f-64a1eadd3a6a', embedding=None, metadata={'rowid': 'AAi0soABZAAAACWAAG', 'dashboard_name': '投控經營分析', 'report_name': '華紙-營運績效分析月報', 'page_name': '事業部層', 'pic_name': '事業部層-全廠損益.jpg', 'description': '這張圖片顯示了一份華紙營運績效分析月報，包含了各事業部的相關分析指標。表格中顯示了以下指標：\n\n**本月**\n- 銷售收入\n- 銷售成本\n- 銷售毛利\n- 營業利益\n- 稅前淨利\n\n**全年累計**\n- 銷售收入\n- 銷售成本\n- 銷售毛利\n- 營業利益\n- 稅前淨利\n- 年度預算\n- 達成率\n\n表格中的商品類別包含：\n- 等等(造紙業)\n- 久違(紙)\n- 花蓮(紙)\n- 台紙(紙)\n- 智慧(紙)\n- P板(紙)\n- 昆山紙業(CNY)\n- 昆林紙業(CNY)\n- 纖維製品(CNY)\n- 合紙(紙)\n\n您可以根據您的需要，查詢您感興趣的指標。\n'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e9a4f944-ea05-4c75-a387-8dc2707e7af0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'rowid': 'AAi0soABZAAAACWAAG', 'dashboard_name': '投控經營分析', 'report_name': '華紙-營運績效分析月報', 'page_name': '事業部層', 'pic_name': '事業部層-全廠損益.jpg', 'description': '這張圖片顯示了一份華紙營運績效分析月報，包含了各事業部的相關分析指標。表格中顯示了以下指標：\n\n**本月**